## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,San Policarpo,PH,12.1791,125.5072,83.44,68,63,6.71,broken clouds
1,1,Rikitea,PF,-23.1203,-134.9692,77.76,71,1,14.12,clear sky
2,2,Lebu,CL,-37.6167,-73.6500,51.58,92,8,11.79,clear sky
3,3,Yellowknife,CA,62.4560,-114.3525,16.32,53,75,5.75,broken clouds
4,4,Carnarvon,AU,-24.8667,113.6333,95.07,28,71,8.05,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,San Policarpo,PH,12.1791,125.5072,83.44,68,63,6.71,broken clouds
1,1,Rikitea,PF,-23.1203,-134.9692,77.76,71,1,14.12,clear sky
7,7,Kismayo,SO,-0.3582,42.5454,82.11,79,85,6.46,overcast clouds
8,8,Avarua,CK,-21.2078,-159.7750,82.45,78,27,3.44,scattered clouds
10,10,Hilo,US,19.7297,-155.0900,79.29,90,75,8.05,broken clouds
19,19,Kapaa,US,22.0752,-159.3190,80.58,85,75,19.57,light rain
25,25,Hithadhoo,MV,-0.6000,73.0833,82.74,69,40,3.33,scattered clouds
30,30,Banes,CU,20.9625,-75.7186,75.88,83,33,9.06,scattered clouds
33,33,Atuona,PF,-9.8000,-139.0333,78.91,78,21,15.95,light rain
44,44,Vaini,TO,-21.2000,-175.2000,84.36,65,75,8.05,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [6]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,San Policarpo,PH,83.44,broken clouds,12.1791,125.5072,
1,Rikitea,PF,77.76,clear sky,-23.1203,-134.9692,
7,Kismayo,SO,82.11,overcast clouds,-0.3582,42.5454,
8,Avarua,CK,82.45,scattered clouds,-21.2078,-159.7750,
10,Hilo,US,79.29,broken clouds,19.7297,-155.0900,
19,Kapaa,US,80.58,light rain,22.0752,-159.3190,
25,Hithadhoo,MV,82.74,scattered clouds,-0.6000,73.0833,
30,Banes,CU,75.88,scattered clouds,20.9625,-75.7186,
33,Atuona,PF,78.91,light rain,-9.8000,-139.0333,
44,Vaini,TO,84.36,broken clouds,-21.2000,-175.2000,


In [7]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head()

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,San Policarpo,PH,83.44,broken clouds,12.1791,125.5072,Agapita M. Nicart
1,Rikitea,PF,77.76,clear sky,-23.1203,-134.9692,People ThankYou
7,Kismayo,SO,82.11,overcast clouds,-0.3582,42.5454,Agan-Hotel
8,Avarua,CK,82.45,scattered clouds,-21.2078,-159.7750,Paradise Inn
10,Hilo,US,79.29,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel


In [8]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City           0
Country        0
Max Temp       0
Description    0
Lat            0
Lng            0
Hotel Name     0
dtype: int64

In [9]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [10]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))